In [1]:
import pandas as pd
from sklearn.preprocessing 
import MinMaxScaler

# Loading the Data
df = pd.read_csv("swiggy_deliveyboy_300.csv", parse_dates=['order_timestamp', 'pickup_timestamp', 'delivery_timestamp'])




In [2]:
#  selecting week from dates
df['delivery_time_mins'] = (df['delivery_timestamp'] - df['pickup_timestamp']).dt.total_seconds() / 60
df['week_num'] = df['order_timestamp'].dt.isocalendar().week - 17
df['year'] = df['order_timestamp'].dt.year



In [3]:
#  Finding First Week Number of the Month
start_date = df['order_timestamp'].min()
first_week = start_date.isocalendar().week 



In [4]:
#  Keeping Only 4 Weeks for a month(starting from first_week)
valid_weeks = [first_week + i for i in range(4)]
df = df[df['order_timestamp'].dt.isocalendar().week.isin(valid_weeks)]


In [5]:
# Weekly Aggregation
weekly_summary = df.groupby(['city', 'year', 'week_num', 'delivery_person_id']).agg(
    total_orders=('order_id', 'count'),
    avg_rating=('order_rating', 'mean'),
    avg_delivery_time=('delivery_time_mins', 'mean')
).reset_index()



In [6]:
#  Normalizing for Score

scaler = MinMaxScaler()
scaled = scaler.fit_transform(weekly_summary[['total_orders', 'avg_rating']])
weekly_summary[['orders_norm', 'rating_norm', 'delivery_time_norm']] = scaled



In [7]:
#  Scoring System (Weights: 0.3, 0.3, 0.4)
weekly_summary['final_score'] = (
    0.30 * weekly_summary['orders_norm'] +
    0.30 * weekly_summary['rating_norm'] +
    0.40 * weekly_summary['delivery_time_norm']
)



In [8]:
# Ranking top 5 per city per week
weekly_summary = weekly_summary.sort_values(by=['city', 'year', 'week_num', 'final_score'], ascending=[True, True, True, False])
weekly_summary['rank'] = weekly_summary.groupby(['city', 'year', 'week_num']).cumcount() + 1

top_5_execs = weekly_summary[weekly_summary['rank'] <= 5]



In [9]:
#  Export to Excel
with pd.ExcelWriter("300_top5_deliveryboy.xlsx") as writer:
    for (year, week), group in top_5_execs.groupby(['year', 'week_num']):
        sheet_name = f"Week_{year}_{week}"
        group_sorted = group.sort_values(by=['city', 'rank'])
        group_sorted.to_excel(writer, sheet_name=sheet_name, index=False)



In [10]:
print(" Excel generated: '300_top5_deliveryboy.xlsx'")

 Excel generated: '300_top5_deliveryboy.xlsx'
